In [64]:
from shiny import App, render, ui, reactive
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer

In [65]:
default_data = pd.read_csv(r'C:\Users\86156\Desktop\lung_disease_data.csv')

In [66]:
# all the clean steps are coded as functions and then add into shiny

In [67]:
## data format clean (standardize string,  convert string into number if avaliable)

In [68]:
def clean_strings_and_convert_numbers(df):
    df_clean = df.copy()
    for col in df_clean.select_dtypes(include=["object"]).columns:
        # remove space and convert to lower
        df_clean[col] = df_clean[col].str.strip().str.lower()
        # remove to np.nan
        df_clean[col] = df_clean[col].replace(["na", "n/a", "none", "null", ""], np.nan)
        # convert to number
        try:
            converted = pd.to_numeric(df_clean[col], errors="coerce")
            if converted.notna().mean() > 0.5:
                df_clean[col] = converted
        except Exception:
            pass
    return df_clean

In [69]:
## convert to datetime

In [70]:
def convert_to_dates(df):
    df_clean = df.copy()
    for col in df_clean.columns:
        if df_clean[col].dtype == "object":
            try:
                converted = pd.to_datetime(df_clean[col], errors="coerce")
                if converted.notna().mean() > 0.5:
                    df_clean[col] = converted
            except Exception:
                pass
    return df_clean

In [71]:
## remove duplicates

In [72]:
def remove_duplicates(df):
    return df.drop_duplicates()

In [73]:
## missing value(able to replace missing value with mean, median, mode or just drop)
## user can handle specific columns also automately handle all numerical columns

In [74]:
def handle_missing_values(df, strategy="mean", drop_columns=[]):
    if len(drop_columns) > 0:
    
        df = df.dropna(subset=drop_columns)
    else:
        if strategy == "drop":
            df = df.dropna() 
        else:
            imputer = SimpleImputer(strategy=strategy)
            num_cols = df.select_dtypes(include=["number"]).columns
            df[num_cols] = imputer.fit_transform(df[num_cols])
    
    return df

In [75]:
## outlier
## function detect outliers using IQR method or z-score method, and can delete, replace as mean or quantiles

In [76]:
def handle_outliers(df, method="IQR", z_thresh=3, columns=None, handling_method="delete"):
    if columns is None:
        numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
    else:
        numeric_cols = [col for col in columns if col in df.columns and pd.api.types.is_numeric_dtype(df[col])]
    if not numeric_cols:
        return df, pd.DataFrame()
    
    # category as methods
    if method == "IQR":
        if handling_method == "delete":
            mask = pd.Series(False, index=df.index)
            for col in numeric_cols:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                mask = mask | ((df[col] < lower_bound) | (df[col] > upper_bound))
            outliers = df[mask]
            df_clean = df[~mask]
            modifications = outliers.copy()
        elif handling_method in ["mean", "median", "winsorize"]:
            df_clean = df.copy()
            modifications = []
            for col in numeric_cols:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                for idx, value in df_clean[col].iteritems():
                    if value < lower_bound or value > upper_bound:
                        original_value = value
                        if handling_method == "mean":
                            replacement = df[col].mean()
                        elif handling_method == "median":
                            replacement = df[col].median()
                        elif handling_method == "winsorize":
                            replacement = lower_bound if value < lower_bound else upper_bound
                        modifications.append({
                            "index": idx,
                            "column": col,
                            "original": original_value,
                            "replacement": replacement
                        })
                        df_clean.at[idx, col] = replacement
            modifications = pd.DataFrame(modifications)
        else:
            df_clean = df.copy()
            modifications = pd.DataFrame()
    elif method == "Z-score":
        if handling_method == "delete":
            mask = pd.Series(False, index=df.index)
            for col in numeric_cols:
                mean = df[col].mean()
                std = df[col].std()
                lower_bound = mean - z_thresh * std
                upper_bound = mean + z_thresh * std
                mask = mask | ((df[col] < lower_bound) | (df[col] > upper_bound))
            outliers = df[mask]
            df_clean = df[~mask]
            modifications = outliers.copy()
        elif handling_method in ["mean", "median", "winsorize"]:
            df_clean = df.copy()
            modifications = []
            for col in numeric_cols:
                mean = df[col].mean()
                std = df[col].std()
                lower_bound = mean - z_thresh * std
                upper_bound = mean + z_thresh * std
                for idx, value in df_clean[col].iteritems():
                    if value < lower_bound or value > upper_bound:
                        original_value = value
                        if handling_method == "mean":
                            replacement = mean
                        elif handling_method == "median":
                            replacement = df[col].median()
                        elif handling_method == "winsorize":
                            replacement = lower_bound if value < lower_bound else upper_bound
                        modifications.append({
                            "index": idx,
                            "column": col,
                            "original": original_value,
                            "replacement": replacement
                        })
                        df_clean.at[idx, col] = replacement
            modifications = pd.DataFrame(modifications)
        else:
            df_clean = df.copy()
            modifications = pd.DataFrame()
    else:
        df_clean = df.copy()
        modifications = pd.DataFrame()
    return df_clean, modifications


In [77]:
## standardize numerical data
## choose specific columns to normalize

In [78]:
def normalize_data(df, normalize_columns=[]):
    if len(normalize_columns) > 0:
        scaler = StandardScaler()
        df[normalize_columns] = scaler.fit_transform(df[normalize_columns])
    return df

In [79]:
## encoding categorical variables
## for variables' unique value lessthan a user set threshold using one-hot encoing, or use lable encoding. default threshold:10

In [80]:
def encode_categorical_data(df, one_hot_threshold=10):
    df = df.copy() 
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns
    for col in categorical_cols:
        unique_values = df[col].nunique()
        # skip if only one unique value
        if unique_values <= 1:
            continue
        
        if unique_values <= one_hot_threshold:
            encoder = OneHotEncoder(sparse_output=False, drop="first")
            encoded_cols = encoder.fit_transform(df[[col]])
            if encoded_cols.shape[1] == 0:
                continue
            encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out([col]), index=df.index)
            df = df.drop(columns=[col])
            df = pd.concat([df, encoded_df], axis=1)
        else:
            from sklearn.preprocessing import LabelEncoder
            encoder = LabelEncoder()
            df[col] = encoder.fit_transform(df[col])
    return df

In [81]:
# shiny ui

In [82]:
app_ui = ui.page_sidebar(
    ui.sidebar(
        ui.input_radio_buttons(
            "data_source", 
            "Choose Data Source: ", 
            choices=["Upload dataset", "Use Default Data"], 
            selected="Use Default Data"
        ),
        ui.input_file("file", "Upload a dataset", multiple=False, accept=[".csv",".rds",".xlsx",".json"]),
        title="Load Data",
    ),
    ui.page_fillable(
        ui.navset_card_tab(
            ui.nav_panel("Data Output", 
                ui.output_table("table")
            ),
            ui.nav_panel("Cleaning & Preprocessing",
                # upper part: different operation columns
                 ui.row(
                    # basic clean (string clean, number convert, duplication remove)
                    ui.column(2,
                        ui.h4("Data Cleaning"),
                        ui.input_checkbox("apply_string_cleaning", "Clean Strings & Convert Numbers", value=False),
                        ui.input_checkbox("apply_date_conversion", "Convert to Dates", value=False),
                        ui.input_checkbox("remove_duplicates", "Remove Duplicate Rows", value=False)
                    ),
                    # missing value
                    ui.column(2,
                        ui.h4("Missing Value Handling"),
                        ui.input_selectize("missing_value_strategy", "Missing Value Strategy:", 
                                             choices=["mean", "median", "mode", "drop"], selected="mean"),
                        ui.input_checkbox("select_all_na", "Select All Columns for NA Removal", value=False),
                        ui.input_checkbox("deselect_all_na", "Deselect All Columns for NA Removal", value=False),
                        ui.input_selectize("drop_na_columns", "Columns to Drop NA:", choices=[], multiple=True)
                    ),
                    # ouliers
                    ui.column(2,
                        ui.h4("Outlier Handling"),
                        ui.input_checkbox("remove_outliers", "Handle Outliers", value=False),
                        ui.input_radio_buttons("outlier_method", "Detection Method:", choices=["IQR", "Z-score"], selected="IQR"),
                        ui.input_slider("zscore_threshold", "Z-score Threshold", min=2, max=5, value=3, step=0.1),
                        ui.input_selectize("outlier_columns", "Columns for Outlier Handling:", choices=[], multiple=True),
                        ui.input_checkbox("select_all_outliers", "Select All Numeric Columns", value=False),
                        ui.input_checkbox("deselect_all_outliers", "Deselect All Numeric Columns", value=False),
                        ui.input_selectize("outlier_handling", "Handling Option:", 
                                           choices=["delete", "mean", "median", "winsorize"], selected="delete")
                    ),
                    # normalize
                    ui.column(2,
                        ui.h4("Normalization"),
                        ui.input_checkbox("enable_normalization", "Enable Normalization", value=False),
                        ui.input_checkbox("select_all_normalize", "Select All Numeric Columns", value=False),
                        ui.input_checkbox("deselect_all_normalize", "Deselect All Numeric Columns", value=False),
                        ui.input_selectize("normalize_columns", "Columns to Normalize:", choices=[], multiple=True)
                    ),
                    # encode
                    ui.column(2,
                        ui.h4("Encoding"),
                        ui.input_checkbox("perform_encoding", "Perform Encoding", value=False),
                        ui.input_slider("one_hot_threshold", "One-Hot Encoding Threshold", min=2, max=50, value=10)
                    )
                ),
                # lower part: left for data preview, right for modifications review
                ui.row(
                    ui.column(6,
                        ui.h4("Data Set Preview"),
                        ui.output_table("preview_table")
                    ),
                    ui.column(6,
                        ui.h4("Modifications (Deleted/Changed Rows)"),
                        ui.output_table("modifications_table")
                    )
                )
            ),
            ui.nav_panel("Feature Engineering", 
                ui.output_table("encoded_table")
            ),
            ui.nav_panel("EDA", "Panel D content"),
            id="tab"
        )
    ),
    title="Team 10- 5243 Project 2",
)


In [83]:
# server 

In [84]:
def server(input, output, session):
    removed_rows = reactive.Value(pd.DataFrame())
    outlier_modifications = reactive.Value(pd.DataFrame())

    @reactive.calc
    def get_data():
        if input.data_source() == "Use Default Data":
            return default_data.copy()
        file = input.file()
        if not file:
            return None  
        return pd.read_csv(file[0]["datapath"])

    @reactive.effect
    def update_column_choices():
        df = get_data()
        if df is not None:
            numeric_columns = df.select_dtypes(include=["number"]).columns.tolist()
            ui.update_selectize("drop_na_columns", choices=df.columns.tolist(), session=session)
            ui.update_selectize("normalize_columns", choices=numeric_columns, session=session)
            ui.update_selectize("outlier_columns", choices=numeric_columns, session=session)

            if input.select_all_na():
                ui.update_selectize("drop_na_columns", selected=df.columns.tolist(), session=session)
            elif input.deselect_all_na():
                ui.update_selectize("drop_na_columns", selected=[], session=session)

            if input.select_all_normalize():
                ui.update_selectize("normalize_columns", selected=numeric_columns, session=session)
            elif input.deselect_all_normalize():
                ui.update_selectize("normalize_columns", selected=[], session=session)

            if input.select_all_outliers():
                ui.update_selectize("outlier_columns", selected=numeric_columns, session=session)
            elif input.deselect_all_outliers():
                ui.update_selectize("outlier_columns", selected=[], session=session)

    @reactive.calc
    def cleaned_data():
        df = get_data()
        if df is None:
            print("⚠ No data to clean")
            return None
        ### standardize data format
        if input.apply_string_cleaning():
            df = clean_strings_and_convert_numbers(df)
        ### date convert
        if input.apply_date_conversion():
            df = convert_to_dates(df)
        ### missing value
        df = handle_missing_values(df, strategy=input.missing_value_strategy(), drop_columns=input.drop_na_columns())

        ### dupulications
        if input.remove_duplicates():
            mask = df.duplicated(keep='first')
            removed = df[mask]
            removed_rows.set(removed)
            df = remove_duplicates(df)
        else:
            removed_rows.set(pd.DataFrame())

        ### outliers
        if input.remove_outliers():
            method = input.outlier_method()
            z_thresh = input.zscore_threshold()
            outlier_cols = list(input.outlier_columns()) if input.outlier_columns() else df.select_dtypes(include=["number"]).columns.tolist()
            handling_option = input.outlier_handling()
            df, modifications = handle_outliers(df, method=method, z_thresh=z_thresh, columns=outlier_cols, handling_method=handling_option)
            outlier_modifications.set(modifications)
        else:
            outlier_modifications.set(pd.DataFrame())

        ### normalize
        if input.enable_normalization():
            normalize_columns = list(input.normalize_columns())
            df = normalize_data(df, normalize_columns=normalize_columns)

        return df

    ### encoding
    @reactive.calc
    def encoded_data():
        df = cleaned_data()
        if df is None:
            return None
        if input.perform_encoding():
            return encode_categorical_data(df, one_hot_threshold=input.one_hot_threshold())
        else:
            return df

    ### combine outliers and dupilications
    @reactive.calc
    def modifications_data():
        dup = removed_rows()
        out_mod = outlier_modifications()
        if dup is None:
            dup = pd.DataFrame()
        if out_mod is None:
            out_mod = pd.DataFrame()
        if dup.empty and out_mod.empty:
            return pd.DataFrame()
        try:
            return pd.concat([dup, out_mod], ignore_index=True)
        except Exception:
            return out_mod

    # data preview
    @output
    @render.table
    def table():
        return get_data()

    @output
    @render.table
    def preview_table():
        df = encoded_data() if input.perform_encoding() else cleaned_data()
        if df is None or df.empty:
            return pd.DataFrame({"Message": ["No data available"]})
        n = len(df)
        if n > 20:
            preview = pd.concat([df.head(10), df.tail(10)])
        else:
            preview = df
        return preview

    ### modifications
    @output
    @render.table
    def modifications_table():
        return modifications_data()

    @output
    @render.table
    def encoded_table():
        return encoded_data()

app = App(app_ui, server)

In [85]:
# run app

In [86]:
import nest_asyncio
nest_asyncio.apply()

from shiny import App
app.run()

INFO:     Started server process [21448]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21448]
